In [1]:
import argparse
from openvino.inference_engine import IECore, IENetwork
import cv2
import numpy as np

/home/ingramai/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.6 of module 'openvino.inference_engine.ie_api' does not match runtime version 3.7
  return f(*args, **kwds)


In [3]:
# load the model to the Inference Engine

def load_model(xml_file, bin_file):
    
    #xml_file = args.xml
    
    #bin_file = args.bin
    
    core = IECore()   #create an instance of the inference engine
    
    network = IENetwork(model=xml_file, weights=bin_file)
    
    # check if any of the network layers is not supported for the target hardware:
    
    suppoted_layers = core.query_network(network = network, device_name = 'CPU') # get supported layers
    
    unsupported_layers = [layer for layer in network.layers.keys() if layer not in suppoted_layers] # get unsupported layers
    
    if len(unsupported_layers) == 0:
        
        print('All network layers are supported!')
    
    else:
    
        print('Those layers are not supported, please add extensions for them', unsupported_layers)
        
    exec_network = core.load_network(network, 'CPU') # load the network to the engine
    
    input_layer = next(iter(network.inputs))   # get input layer of the network
    
    input_shape = network.inputs[input_layer].shape  # get input shape for preprocessing
        
    return exec_network, input_shape
    

In [4]:
def preprocessing(image, input_shape):
    im = np.copy(image)
    im = cv2.resize(im, (input_shape[2], input_shape[3]))
    im = im.transpose((2,0,1))
    im = im.reshape(1,*im.shape)
    
    return im

In [5]:
def sync_inference(executable_network, inputs):
    
    outputs = executable_network.infer(inputs)
    
    return outputs

In [6]:
def async_inference(executable_network, inputs):
    
    handler = executable_network.start_async(0, inputs) 
    
    handler.wait(-1)
    
    outputs = handler.outputs
    
    return outputs

In [7]:
image = cv2.imread('input.jpeg')

emotions = ['neutral', 'happy', 'sad', 'surprise', 'anger']

exec_net, input_shape = load_model('intel/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.xml', 
                      'intel/emotions-recognition-retail-0003/FP32/emotions-recognition-retail-0003.bin')

im = preprocessing(image, input_shape)

input_layer = next(iter(exec_net.inputs))

inputs = {input_layer: im}

sync_output = sync_inference(exec_net, inputs)

emotion = np.argmax(sync_output)

emotion = emotions[emotion]

print('sync inference done!:', emotion)

async_ouput = async_inference(exec_net, inputs)
                   
emotion = np.argmax(async_ouput)

emotion = emotions[emotion]

print('Async inference done!:', emotion)


All network layers are supported!
sync inference done!: neutral
Async inference done!: neutral
